In [ ]:
!pip install torch
!pip install python-dotenv
!pip install mysqlclient
!pip install --pre deepchem[tensorflow]
!pip install --upgrade scikit-learn
!pip install paramiko
!pip install ipython-sql
!pip install tensorflow
!pip install rdkit
!pip install seaborn

In [ ]:
import pandas as pd
import os
import requests
import json
import csv
import io
import re
from collections import defaultdict
from time import time
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sqlalchemy import create_engine
from sqlalchemy import text
import paramiko
import sql
from dotenv import dotenv_values
import deepchem as dc
import numpy as np
import tensorflow as tf
from tensorflow import keras
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import seaborn
from matplotlib.pyplot import hist
from sklearn.neighbors import KernelDensity
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from sklearn.feature_extraction.text import TfidfTransformer
from typing import List

In [ ]:
# Loading the database config values for paramiko
# paramiko connects to jean dubois VM with SSH

config = dotenv_values('database_creds.env')

filename = config['FILENAME']
hostname = config['HOSTNAME']
username = config['USERNAME']
port = int(config['PORT'])
password = config['PASSWORD']
database = config['DATABASE']

dubois_username = config['DUBOIS_USERNAME']
dubois_host = config['DUBOIS_HOST']

# setup client for SSH
client = paramiko.SSHClient()
key = paramiko.RSAKey.from_private_key_file(filename, password=password)
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

print("Connecting to SSH Client...")

try:
    client.connect(hostname, port, username, password=password, pkey=key)
except Exception as e:
    print(e)

print("Successfully connected to SSH Client!")


In [ ]:


# execute commands on jean dubois VM
def exec_cmd(cmd: str, show_output=False):
    stdin, stdout, stderr = client.exec_command(cmd)

    if show_output:
        print(stdout.read().decode())
        print(stderr.read().decode())

    return stdin, stdout, stderr


# convert the CSV data returned by dubois back to a DataFrame (pandas)
def csv_to_df(csv_data: str):
    return pd.read_csv(io.StringIO(csv_data), header=0)


# sends SELECT queries to the database provided and defaults to * if no columns are provided
def query_db(db: str, columns: List[str] = None):
    # formats column names to "x, y, z"
    formatted_columns = re.sub(r"[\[\]']", "", str(columns))

    # python cmd send to dubois
    # imports sqlalchemy and pandas
    # pandas reads the SQL and turns it into a DataFrame object
    # converts DataFrame into csv before printing
    multi_line_cmd = f"""
import sqlalchemy
from sqlalchemy import text
import pandas as pd

engine = sqlalchemy.create_engine('mysql://{dubois_username}:{password}@{dubois_host}/{database}')
with engine.begin() as conn:
    query = text('SELECT {"*" if columns is None else formatted_columns} FROM {db}')
    data = pd.read_sql(query, conn).to_csv(index=False)
    print(data)

exit()
    """

    # reads the output from print statement (csv data from the database)
    std_in, stdout, stderr = exec_cmd(f'python3 -c "{multi_line_cmd}"', False)

    return csv_to_df(stdout.read().decode())


# queries database for matching disease_id
def query_db_for_disease_by_id(db: str, disease_id: str):
    query_data = query_db(db)
    return query_data[query_data.disease_id == disease_id]


# queries database for matching target_ensemble_id
def query_db_for_target_by_id(db: str, target_id: str):
    query_data = query_db(db)
    return query_data[query_data.target_ensemble_id == target_id]


In [ ]:

# Set disease_id variable for desired disease
#at somepoint include api to chose what disease one wants from : https://www.ebi.ac.uk/efo/
disease_id = "EFO_0005537"

disease_df = query_db_for_disease_by_id(db="disease_to_target", disease_id=disease_id)


#displaying the dataframe
display(disease_df)
target_ids = disease_df.sort_values(by = ['association_score'], ascending=False)['target_ensemble_id'].values


In [ ]:


# example target_id from target_to_compounds database
target_id = "ENSG00000196230"

# gets the compounds from the target
target_to_compounds_df = query_db_for_target_by_id(db="target_to_compounds", target_id=target_id)


#show dataframe
display(target_to_compounds_df)

In [ ]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda

In [ ]:
# 1. Train Model
print(tf.version.VERSION)

compound_dataset = target_to_compounds_df
smiles = compound_dataset['smiles']
IC50 = compound_dataset['standard_value']
featurizer = dc.feat.ConvMolFeaturizer()
compound_dataset['featurized'] = featurizer.featurize(smiles)
compound_dataset['divided values'] = compound_dataset['standard_value'].astype(float).div(max(compound_dataset['standard_value'].astype(float)))
compound_dataset['pIC50'] = np.log10(compound_dataset['divided values'].astype(float)).mul(-1)
compound_dataset['number'] = list(range(0,len(compound_dataset)))
display(compound_dataset)

training_dataset = compound_dataset.sample(frac = 0.7)
testing_dataset = (compound_dataset[~compound_dataset['number'].isin(training_dataset['number'])])
display(testing_dataset)

numpy_training_dataset = dc.data.NumpyDataset(X=training_dataset['featurized'],y=training_dataset['pIC50'].astype(float), ids=training_dataset['smiles'])
numpy_testing_dataset = dc.data.NumpyDataset(X=testing_dataset['featurized'],y=testing_dataset['pIC50'].astype(float), ids=testing_dataset['smiles'])
display(numpy_training_dataset)
display(numpy_testing_dataset)
#using  the graph convolution model
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10)
model.fit(numpy_training_dataset, nb_epoch=1)#was 100 on instance

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(numpy_training_dataset, [metric]))
print("Test set score:", model.evaluate(numpy_testing_dataset, [metric]))
#end of Training the model


#2. Download all compounds in database
compounds_df = query_db("target_to_compounds", ["compound_id", "smiles"])

predict_list = []
for smile in range(0, len(compounds_df)):
    predict_list.append(None)

compounds_df['predicted_pIC50'] = predict_list

curated_compounds_df = compounds_df.dropna(subset=['smiles'])

new_smiles = curated_compounds_df['smiles']
curated_compounds_df['featurized'] = featurizer.featurize(new_smiles)

#3. run the model to predict ic50 values for each compound in the database
dataset = dc.data.NumpyDataset(X=curated_compounds_df['featurized'], y=curated_compounds_df['predicted_pIC50'], ids=curated_compounds_df['smiles'])
curated_compounds_df['predicted_pIC50'] = model.predict(dataset)

# 4. displays a dataframe that has the following two columns : compound, predicted ic50
display(curated_compounds_df)

In [ ]:
# 5. print out the compound that has the highest ic50 value that has NOT been tested on the target already.

curated_compounds_no_target = (curated_compounds_df[~curated_compounds_df['smiles'].isin(target_to_compounds_df['smiles'])])

df = curated_compounds_no_target.sort_values(by = ['predicted_pIC50'], ascending=False)
df.drop_duplicates(subset = 'smiles', inplace = True)

display(df)
compound_ids = df['compound_id'].values
compound_smiles = df['smiles'].values

compound_ids_and_smiles = list(zip(compound_ids, compound_smiles))
best_compound = (compound_ids_and_smiles[0])

# 6. get InChI key for top compound and generate a url for Zinc

url = "https://cactus.nci.nih.gov/chemical/structure/{smiles}/stdinchikey".format(smiles = best_compound[1])
r = requests.get(url=url)
raw_inchikey = r.text
inchikey = raw_inchikey.split('=')[1]

print(inchikey)
#zinc website was not working when we checked last
zinc_url = 'https://zinc15.docking.org/substances/?inchikey={inchikey}'.format(inchikey = inchikey)
print(zinc_url)

In [ ]:
small_df = df.head(10)
dataset = dc.data.NumpyDataset(X=small_df['featurized'],y=small_df['predicted_pIC50'].astype(float), ids=small_df['smiles'])

In [ ]:
just_smiles_df = pd.DataFrame()
just_smiles_df['smiles'] = small_df['smiles']
smiles = just_smiles_df['smiles'].tolist
small_smiles = just_smiles_df.head(10)
just_smiles_df['name'] = just_smiles_df['smiles']
just_smiles_df.to_csv('smiles.csv', index = False)

In [ ]:
#making the smiles dataframe
smiles = small_df['smiles']
featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation = True)
small_df['frag_featurized'] = featurizer.featurize(smiles)
display(small_df)

In [ ]:
frag_dataset = dc.data.NumpyDataset(X=small_df['frag_featurized'], y = None, w = None, ids = dataset.ids)

tr = dc.trans.FlatteningTransformer(frag_dataset) # flatten dataset and add ids to each fragment
frag_dataset = tr.transform(frag_dataset)

In [ ]:
# whole molecules
pred = model.predict(dataset)
pred = pd.DataFrame(pred, index=dataset.ids, columns=["Molecule"])  # turn to dataframe for convenience
display(pred)
# fragments
pred_frags = model.predict(frag_dataset)
pred_frags = pd.DataFrame(pred_frags, index=frag_dataset.ids, columns=["Fragment"])  # turn to dataframe for convenience

print(pred_frags)
# merge 2 dataframes by molecule names
mol_df = pd.merge(pred_frags, pred, right_index=True, left_index=True)
# find contribs
mol_df['Contrib'] = mol_df["Molecule"] - mol_df["Fragment"]
display(mol_df)

In [ ]:
def vis_contribs(mols, df, smi_or_sdf = "smi"):
    # input format of file, which was used to create dataset determines the order of atoms,
    # so we take it into account for correct mapping!
    maps = []
    for mol  in mols:
        wt = {}
        if smi_or_sdf == "smi":
            print(mol)
            for n,atom in enumerate(Chem.rdmolfiles.CanonicalRankAtoms(mol)):
                wt[atom] = df.loc[mol.GetProp("_Name"),"Contrib"][n]

        if smi_or_sdf == "sdf":
            for n,atom in enumerate(range(mol.GetNumHeavyAtoms())):
                wt[atom] = df.loc[Chem.MolToSmiles(mol),"Contrib"][n]
        maps.append(SimilarityMaps.GetSimilarityMapFromWeights(mol,wt))
    return maps

In [ ]:
mols = [m for m in Chem.SmilesMolSupplier('smiles.csv', ',') if m is not None]
print(mols)

In [ ]:
vis_contribs(mols, mol_df, 'smi')

In [ ]:


model.save_checkpoint()


In [ ]:
"""
insert code here that
1. retrieves the terms for all the assays that are relevant to the target the user picked.
2. clusters the assays according to their descriptive terms
3. plots the clusters (set n_clusters = 10)
4. prints out the title of one assay from each cluster.
"""
#getting information from the database
config = dotenv_values("database_URL.env")
url = config['DATABASE_URL']
engine = create_engine(url, echo=False)

with engine.begin() as conn:
    query = text("select * from target_to_compounds;")
    target_list = pd.read_sql(query, conn)
display(target_list)


three_col_list = target_list[['assay_id', 'assay_description', 'abstract']].copy()
#getting just the Assay descriptions to prep for clustering
unique_list = []
for index, row in three_col_list.iterrows():
    if (row["assay_id"], row["assay_description"], row["abstract"]) in unique_list:
        continue
    else:
        unique_list.append((row["assay_id"], row["assay_description"], row["abstract"]))

Assay_Descriptions = [abstract for (assay_ids, assay_name, abstract) in unique_list]
Assay_Descriptions_Joined = ':: '.join(Assay_Descriptions)
#making the descriptions into one list
Assay_Descriptions_List = Assay_Descriptions
#the countVectorizer needed for the other vectorizer
Assay_Count_Vect = CountVectorizer()
Assay_Train_Counts = Assay_Count_Vect.fit_transform(Assay_Descriptions_List)
n_clusters = 20
X = Assay_Train_Counts.toarray()
#the second vectorizier (tfidf transformer)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(Assay_Train_Counts)
tf_idf_vector = tfidf_transformer.transform(Assay_Train_Counts)

ward = AgglomerativeClustering(
    n_clusters = n_clusters, linkage="ward", connectivity=None, compute_full_tree= True,compute_distances= True
)
ward.fit(X)

unique_labels, counts = np.unique(ward.labels_, return_counts=True)
cluster_sizes = dict(zip(unique_labels, counts))

largest_cluster_label = max(cluster_sizes, key=cluster_sizes.get)
largest_cluster_size = cluster_sizes[largest_cluster_label]

print(f"largest cluster: Cluster {largest_cluster_label}, Size: {largest_cluster_size}")

cluster_labels = ward.fit_predict(tf_idf_vector.toarray())

cluster_assay_descriptions = {}
for cluster_label, assay_description in zip(cluster_labels, Assay_Descriptions):
    if cluster_label not in cluster_assay_descriptions:
        cluster_assay_descriptions[cluster_label] = []
    cluster_assay_descriptions[cluster_label].append(assay_description)

sorted_clusters = sorted(cluster_assay_descriptions.keys())
#getting and showing the cluster labels
for cluster_label in sorted_clusters:
    assay_descriptions = cluster_assay_descriptions[cluster_label]
    print(f"Cluster {cluster_label}: {assay_descriptions[0]}")

X_Hist = ward.distances_
kernel = stats.gaussian_kde(X_Hist)
print(kernel(X_Hist))

df_idf = pd.DataFrame(tfidf_transformer.idf_,index=Assay_Count_Vect.get_feature_names_out(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights'])

count_vector=Assay_Count_Vect.transform(Assay_Descriptions_List)

tf_idf_vector=tfidf_transformer.transform(count_vector)
print(tf_idf_vector)

feature_names = Assay_Count_Vect.get_feature_names_out()

first_document_vector=tf_idf_vector[0]

df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)
#showing the Graph
seaborn.clustermap(tf_idf_vector.toarray(),method='ward')

print(len(cluster_labels))
print(len(Assay_Descriptions))
print(cluster_assay_descriptions[4][0] == cluster_assay_descriptions[4][1])
print(cluster_assay_descriptions[4][9])